# Jupyter Notebook for Calculating Statistics

In [ ]:
model_metadata = {
    "alpacare-7B": {"model_type": "clinical/biomedical", "model_size_in_b": 7},
    "alpacare-13B": {"model_type": "clinical/biomedical", "model_size_in_b": 13},
    "biomedgpt7B": {"model_type": "clinical/biomedical", "model_size_in_b": 7},
    "biomistral7B": {"model_type": "clinical/biomedical", "model_size_in_b": 7},
    "claude_3.5-haiku": {"model_type": "closed/proprietary", "model_size_in_b": None}, # 175?
    "claude_3.5-sonnet": {"model_type": "closed/proprietary", "model_size_in_b": None}, # 175?
    "gemini_1.5_flash": {"model_type": "closed/proprietary", "model_size_in_b": None},
    "gemini_1.5_flash-8B": {"model_type": "closed/proprietary", "model_size_in_b": 8},
    "gpt4o": {"model_type": "closed/proprietary", "model_size_in_b": None}, # around 1 trillion (1000B)
    "gpt4o-mini": {"model_type": "closed/proprietary", "model_size_in_b": None}, # 175?
    "gpt35": {"model_type": "closed/proprietary", "model_size_in_b": 175},
    "llama2_chat-7B": {"model_type": "open - general", "model_size_in_b": 7},
    "llama2_chat-13B": {"model_type": "open - general", "model_size_in_b": 13},
    "llama2_chat-70B": {"model_type": "open - general", "model_size_in_b": 70},
    "llama3_instruct-8B": {"model_type": "open - general", "model_size_in_b": 8},
    "llama3_instruct-70B": {"model_type": "open - general", "model_size_in_b": 70},
    "med42-8B": {"model_type": "clinical/biomedical", "model_size_in_b": 8},
    "med42-70B": {"model_type": "clinical/biomedical", "model_size_in_b": 70},
    "mistral_instruct7B": {"model_type": "open - general", "model_size_in_b": 7},
    "olmo2_instruct-7B": {"model_type": "open - general", "model_size_in_b": 7},
    "olmo2_instruct-13B": {"model_type": "open - general", "model_size_in_b": 13},
    "openbiollm-8B": {"model_type": "clinical/biomedical", "model_size_in_b": 8},
    "openbiollm-70B": {"model_type": "clinical/biomedical", "model_size_in_b": 70}
}

## Spin Detection Task

Average of accuracy, precision, recall, and F1 score by model type

## RCT Trial Result Interpretation Task